In [42]:
import pandas as pd
import nameparser
import json
import requests
import numpy as np

pro_publica_api_key = "PguHImFJakafbsBpZ2ed21A1HIgvXKiM7tg0BqI0 "
headers = {'X-API-Key': pro_publica_api_key}

In [54]:
bills = pd.read_csv('../prod_data/member_bills.csv')
bills['bill_intro_date'] = pd.to_datetime(bills['bill_intro_date'])
bills = bills[(bills['bill_intro_date'] > '2019-01-01') & (bills['bill_intro_date'] < '2023-05-03')]

bills_id = bills[['bill_slug','member_id', 'bill_congress']]

bills_id

,bill_slug,member_id,bill_congress
20,s1194,C000174,118
21,sres143,C000174,118
22,s654,C000174,118
23,sres33,C000174,118
24,s51,C000174,118
...,...,...,...
18635,hr1715,M001222,118
18636,hres76,M001222,118
18641,hr2597,J000308,118
18650,hr2810,T000488,118


In [56]:
related_bills_old = pd.read_csv('../prod_data/related_bills.csv')
related_bills_old


,related_bill_id,related_bill_title,related_bill_introduction_date,bill_slug,bill_congress,member_id
0,hr6056-118,To direct the Secretary of Defense to submit t...,2023-10-25,hr6056,118,L000590
1,hr5997-118,To require the Secretary of Energy to further ...,2023-10-19,hr5997,118,L000590
2,hr5720-118,To assist trafficking survivors who lack gover...,2023-09-26,hr5720,118,L000590
3,hr5645-118,To establish a matched savings program for low...,2023-09-21,hr5645,118,L000590
4,hr5598-118,To amend the SUPPORT for Patients and Communit...,2023-09-20,hr5598,118,L000590
...,...,...,...,...,...,...
4897,hr3155-118,"To amend title 18, United States Code, to proh...",2023-05-09,hr3155,118,M001217
4898,hr4073-118,To amend the Harmonized Tariff Schedule of the...,2023-06-13,hr4073,118,M001220
4899,hres76-118,Removing a certain Member from a certain stand...,2023-01-31,hres76,118,M001222
4900,hr2597-118,To authorize members of reserve components of ...,2023-04-13,hr2597,118,J000308


In [57]:
related_bills_cache = pd.read_csv('related_bills.csv')

related_bills_cache

,related_bill_id,related_bill_title,related_bill_introduction_date,bill_slug,bill_congress
0,s51-118,A bill to provide for the admission of the Sta...,2023-01-24,s51,118
1,s5106-117,A bill to amend title XVIII of the Social Secu...,2022-11-16,s5106,117
2,s5011-117,A bill to amend title XIX of the Social Securi...,2022-09-29,s5011,117
3,sconres46-117,A concurrent resolution commemorating the 50th...,2022-09-29,sconres46,117
4,s4194-117,A bill to amend the Fish and Wildlife Act of 1...,2022-05-12,s4194,117
...,...,...,...,...,...
4656,s147-116,"A bill to amend title 31, United States Code, ...",2019-01-16,s147,116
4657,s148-116,A bill to require a full audit of the Board of...,2019-01-16,s148,116
4658,s158-116,A bill to prohibit Federal funding of Planned ...,2019-01-16,s158,116
4659,s159-116,A bill to implement equal protection under the...,2019-01-16,s159,116


In [59]:
related_bills_cache= related_bills_cache.merge(bills_id, how='left', on=['bill_slug', 'bill_congress'])

In [61]:
related_bills_final = pd.concat([related_bills_old, related_bills_cache])

related_bills_final.to_csv('../prod_data/related_bills.csv', index=False)

In [39]:
bills_covered = related_bills_old['bill_slug'].unique()

bills_of_interest = bills[~bills['bill_slug'].isin(bills_covered)]

bills_of_interest

,bill_congress,bill_id,bill_title,bill_intro_date,member_id,bill_summary,bill_url,bill_slug
20,118,s1194-118,A bill to require the Administrator of the Env...,2023-04-19,C000174,NaN,https://www.congress.gov/bill/118th-congress/s...,s1194
21,118,sres143-118,A resolution expressing support for the design...,2023-03-30,C000174,NaN,https://www.congress.gov/bill/118th-congress/s...,sres143
22,118,s654-118,A bill to amend the Water Infrastructure Impro...,2023-03-06,C000174,NaN,https://www.congress.gov/bill/118th-congress/s...,s654
23,118,sres33-118,An original resolution authorizing expenditure...,2023-02-07,C000174,NaN,https://www.congress.gov/bill/118th-congress/s...,sres33
24,118,s51-118,A bill to provide for the admission of the Sta...,2023-01-24,C000174,NaN,https://www.congress.gov/bill/118th-congress/s...,s51
...,...,...,...,...,...,...,...,...
18598,116,hr4076-116,To require the Securities and Exchange Commiss...,2019-07-25,W000812,Modernizing Disclosures for Investors Act This...,https://www.congress.gov/bill/116th-congress/h...,hr4076
18634,118,hres352-118,Expressing the sense of the House of Represent...,2023-05-02,M001222,NaN,https://www.congress.gov/bill/118th-congress/h...,hres352
18635,118,hr1715-118,To direct the Department of Energy and the Nat...,2023-03-22,M001222,NaN,https://www.congress.gov/bill/118th-congress/h...,hr1715
18650,118,hr2810-118,To designate the Federal building located at 9...,2023-04-24,T000488,NaN,https://www.congress.gov/bill/118th-congress/h...,hr2810


In [40]:
bill_slug_list = []
bill_congress_list = []
related_bill_id_list = []
related_bill_title_list = []
related_bill_introduction_date_list = []
iteration_count = 0
save_interval = 30
for index, row in bills_of_interest.iterrows():
    iteration_count +=1
    congress = row['bill_congress']
    bill_id = row['bill_slug']
    

    url = f"https://api.propublica.org/congress/v1/{congress}/bills/{bill_id}/related.json"
    response = requests.get(url, headers=headers)
    if 'results' in response.json():
        if response.json()['num_results'] >0:
            results = response.json()['results']
            for result in results:
                bill_slug_list.append(bill_id)
                bill_congress_list.append(congress)
                related_bill_id_list.append(result['bill_id'])
                related_bill_title_list.append(result['title'])
                related_bill_introduction_date_list.append(result['introduced_date'])
        else:
            pass
        
    else:
        pass
    
    if iteration_count % save_interval == 0:
        temp_df =pd.DataFrame({'related_bill_id':related_bill_id_list,
                               'related_bill_title':related_bill_title_list,
                               'related_bill_introduction_date':related_bill_introduction_date_list,
                               'bill_slug':bill_slug_list,
                               'bill_congress':bill_congress_list})

        temp_df.to_csv('related_bills.csv', index=False)
        print(f"Saved {iteration_count} records")



Saved 30 records
Saved 60 records
Saved 90 records
Saved 120 records
Saved 150 records
Saved 180 records
Saved 210 records
Saved 240 records
Saved 270 records
Saved 300 records
Saved 330 records
Saved 360 records
Saved 390 records
Saved 420 records
Saved 450 records
Saved 480 records
Saved 510 records
Saved 540 records
Saved 570 records
Saved 600 records
Saved 630 records
Saved 660 records
Saved 690 records
Saved 720 records
Saved 750 records
Saved 780 records
Saved 810 records
Saved 840 records
Saved 870 records
Saved 900 records
Saved 930 records
Saved 960 records
Saved 990 records
Saved 1020 records
Saved 1050 records
Saved 1080 records
Saved 1110 records
Saved 1140 records
Saved 1170 records
Saved 1200 records
Saved 1230 records
Saved 1260 records
Saved 1290 records
Saved 1320 records
Saved 1350 records
Saved 1380 records
Saved 1410 records
Saved 1440 records
Saved 1470 records
Saved 1500 records
Saved 1530 records
Saved 1560 records
Saved 1590 records
Saved 1620 records
Saved 1650

KeyboardInterrupt: 

In [41]:
related_bills = pd.read_csv('related_bills.csv')
related_bills

,related_bill_id,related_bill_title,related_bill_introduction_date,bill_slug,bill_congress
0,s51-118,A bill to provide for the admission of the Sta...,2023-01-24,s51,118
1,s5106-117,A bill to amend title XVIII of the Social Secu...,2022-11-16,s5106,117
2,s5011-117,A bill to amend title XIX of the Social Securi...,2022-09-29,s5011,117
3,sconres46-117,A concurrent resolution commemorating the 50th...,2022-09-29,sconres46,117
4,s4194-117,A bill to amend the Fish and Wildlife Act of 1...,2022-05-12,s4194,117
...,...,...,...,...,...
4656,s147-116,"A bill to amend title 31, United States Code, ...",2019-01-16,s147,116
4657,s148-116,A bill to require a full audit of the Board of...,2019-01-16,s148,116
4658,s158-116,A bill to prohibit Federal funding of Planned ...,2019-01-16,s158,116
4659,s159-116,A bill to implement equal protection under the...,2019-01-16,s159,116


In [25]:

related_bills.to_csv('../prod_data/related_bills.csv', index=False)


In [26]:
related_bills

,related_bill_id,related_bill_title,related_bill_introduction_date,bill_slug,bill_congress
0,hr6056-118,To direct the Secretary of Defense to submit t...,2023-10-25,hr6056,118
1,hr5997-118,To require the Secretary of Energy to further ...,2023-10-19,hr5997,118
2,hr5720-118,To assist trafficking survivors who lack gover...,2023-09-26,hr5720,118
3,hr5645-118,To establish a matched savings program for low...,2023-09-21,hr5645,118
4,hr5598-118,To amend the SUPPORT for Patients and Communit...,2023-09-20,hr5598,118
...,...,...,...,...,...
4897,hr3155-118,"To amend title 18, United States Code, to proh...",2023-05-09,hr3155,118
4898,hr4073-118,To amend the Harmonized Tariff Schedule of the...,2023-06-13,hr4073,118
4899,hres76-118,Removing a certain Member from a certain stand...,2023-01-31,hres76,118
4900,hr2597-118,To authorize members of reserve components of ...,2023-04-13,hr2597,118
